In [1]:
import numpy as np
import pandas as pd
import json

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold

import matplotlib.pyplot as plt
# import plotly.graph_objs as go
# import plotly.plotly as py
# import plotly.io as pio

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic,
                                              ExpSineSquared, DotProduct,
                                              ConstantKernel)
from sklearn.model_selection import train_test_split

%pylab inline
import GPy
import GPyOpt
from numpy.random import seed

Populating the interactive namespace from numpy and matplotlib


1. https://www.blopig.com/blog/wp-content/uploads/2019/10/GPyOpt-Tutorial1.html

2. https://nbviewer.org/github/SheffieldML/GPyOpt/blob/devel/manual/GPyOpt_constrained_optimization.ipynb

3. https://gpyopt.readthedocs.io/en/latest/GPyOpt.core.html

In [2]:
Factors_Bounds = pd.read_excel('cytokinesInBlood.xlsx')
N_factors = Factors_Bounds.shape[0]
print(Factors_Bounds)

  Cytokine  Concentration in Serum (pg/mL)
0     IL-7                             2.8
1     IL-4                             8.8
2     IL-3                            12.0
3    IL-21                            62.6
4     IL-2                             4.2
5    IL-15                             2.1
6     BAFF                             1.0
7    IL-12                            41.0


In [3]:
bounds = []
for i in range(N_factors):
    temp_dict = {'name': Factors_Bounds["Cytokine"].iloc[i], 'type': 'continuous', 
                 'domain': (0, 2 * Factors_Bounds["Concentration in Serum (pg/mL)"].iloc[i])}
    bounds.append(temp_dict)

### Initial Design

In [4]:
random.seed(37)

feasible_region = GPyOpt.Design_space(space = bounds) 
initial_design = GPyOpt.experiment_design.initial_design('latin', feasible_region, 6)

In [5]:
pd.DataFrame(initial_design, columns = Factors_Bounds["Cytokine"].to_list())

,IL-7,IL-4,IL-3,IL-21,IL-2,IL-15,BAFF,IL-12
0,3.266667,16.133333,2.0,52.166667,4.9,2.45,1.833333,6.833333
1,1.400000,10.266667,14.0,114.766667,7.7,1.75,0.166667,34.166667
2,2.333333,1.466667,10.0,10.433333,2.1,0.35,1.500000,20.500000
3,4.200000,4.400000,6.0,73.033333,0.7,3.15,1.166667,61.500000
4,5.133333,13.200000,18.0,31.300000,3.5,3.85,0.500000,75.166667
5,0.466667,7.333333,22.0,93.900000,6.3,1.05,0.833333,47.833333


In [6]:
pd.DataFrame(initial_design, columns = Factors_Bounds["Cytokine"].to_list()).to_csv('0_Cytokine_ExperimentalPlan.csv')